# ALL installs 

In [1]:
# ! pip install requests"
# ! pip install kafka-python
# ! pip install pyspark
# !pip  install -r requirements.txt

In [2]:
!pip freeze > requirements.txt

# ALL imports

In [3]:
from allsecrets import *
from titles import *
from coordinates import *
import json,requests
from json import dumps,loads 
from kafka import KafkaProducer, KafkaConsumer


In [4]:
from pyspark.sql import SparkSession , functions as F

from pyspark.sql.functions import udf,col,countDistinct,date_format,row_number

from pyspark.sql.types import FloatType , StringType , IntegerType , StructType , StructField , TimestampType

from pyspark.sql.window import Window 

spark = SparkSession.builder.appName('Weather')\
    .getOrCreate()



your 131072x1 screen size is bogus. expect trouble


22/11/17 16:04:44 WARN Utils: Your hostname, AMRIT resolves to a loopback address: 127.0.1.1; using 172.20.115.244 instead (on interface eth0)
22/11/17 16:04:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/17 16:04:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Json print function

In [5]:
def json_print(json_object):
    text = json.dumps(json_object, sort_keys=True, indent=4) # sort_keys=True means sort the keys in alphabetical order
    # indent 4 means 4 spaces for each indentation
    print(text)

## Openweather 5 days 3 hour forcast

In [6]:

# parameters = {'lat': '27.7167', "lon": '85.3667', "appid": open_weather_api}
# sample_response = requests.get("http://api.openweathermap.org/data/2.5/forecast", params=parameters)

# # save the response as json file
# with open('sample_response.json', 'w') as f:
#     json.dump(sample_response.json(), f)

# weather_rdd= spark.sparkContext.parallelize([json.dumps(sample_response.json()) ]) # convert to rdd

# #convert to spark dataframe
# weather_df = spark.read.json(weather_rdd) # convert to spark dataframe

# schema_weather = weather_df.schema # get the schema of the dataframe

# #  save  schema_weather  to .json file
# with open('schema_weather.json', 'w') as f:
#     json.dump(schema_weather.jsonValue(), f, indent=4)

# #  load  schema_weather  from .json file
# with open('schema_weather.json', 'r') as f:
#     schema_weather1 = F.StructType.fromJson(json.load(f)) 

# # schema_weather1

# # weather_df.printSchema()


In [7]:
# #api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}&appid={API key}

# for lat ,lon in lat_lon.items():
#     parameters = {'lat': lat, "lon": lon, "appid": open_weather_api}
#     weather_response = requests.get("http://api.openweathermap.org/data/2.5/forecast", params=parameters)
#     print (weather_response.url)
#     print(weather_response.status_code)
#     # json_print(weather_response.json())



# Test produces and consumer

## Test producer

In [11]:
producer = KafkaProducer(bootstrap_servers=[
                         'localhost:9092'], value_serializer=lambda x: dumps(x).encode('utf-8'))
                         # both key(header) and value are converted to json format

for lat, lon in lat_lon.items():
    parameters = {'lat': lat, "lon": lon, "appid": open_weather_api}
    weather_response = requests.get(
        "http://api.openweathermap.org/data/2.5/forecast", params=parameters)

    producer.send('weather', value=weather_response.json())
    producer.flush()  # flush the data to the kafka broker ( topic) and  make sure data  is sent to the kafka broker and  not lost in the buffer

## Test consumer

In [15]:

Consumer = KafkaConsumer('weather', bootstrap_servers=['localhost:9092'], auto_offset_reset='earliest',
                         enable_auto_commit=True, value_deserializer=lambda x: loads(x.decode('utf-8')))

In [17]:
# consume the data from the kafka broker and save to json file   %%%% stop it after some time   10-15 sec %%%%%
for message in Consumer:
    message = message.value
    with open('weather.json', 'w') as f:
        json.dump(message, f)
        f.write(' ') # add space between each json object


KeyboardInterrupt: 

In [66]:
# close consumer to free up resources after reading the messages
# Consumer.close()

In [35]:
Consumer.topics()

{'_confluent-command',
 '_confluent-telemetry-metrics',
 '_confluent_balancer_api_state',
 '_confluent_balancer_broker_samples',
 '_confluent_balancer_partition_samples',
 '_schemas',
 'connect-configs',
 'connect-offsets',
 'connect-status',
 'weather'}

In [34]:
# create topic
! kafka-topics --bootstrap-server localhost:9092 --create --topic weather --replication-factor 1 --partitions 1



Error while executing topic command : Topic 'weather' already exists.
[2022-11-17 16:29:40,109] ERROR org.apache.kafka.common.errors.TopicExistsException: Topic 'weather' already exists.
 (kafka.admin.TopicCommand$)


In [149]:
# delete topic
! kafka-topics --bootstrap-server localhost:9092 --delete --topic cleaned_table

# Spark magic

## Unpacking and Exploding the  columns

In [224]:

weather_df = spark.read.json('weather.json')

nested_json_type=type (weather_df.select("city").take(1)[0][0]) # get the type of the nested json

# function to expand the nested json object only
def expand_json(df):
    for c in df.columns:
        if type(df.select(c).take(1)[0][0]) == nested_json_type:
            df = df.select("*", F.col(c+".*")).drop(c)
            print(c)
    return df
    
# weather_df.printSchema()

In [225]:

#explode "list" column  as it is in list  of dictionary conataing 40 records
weather_df = weather_df.select ("*", F.explode("list").alias("every_3_hr_weather")).drop("list")

weather_df.show(5)

+--------------------+---+---+-------+--------------------+
|                city|cnt|cod|message|  every_3_hr_weather|
+--------------------+---+---+-------+--------------------+
|{{27.7167, 85.366...| 40|200|      0|{{42}, 1668686400...|
|{{27.7167, 85.366...| 40|200|      0|{{37}, 1668697200...|
|{{27.7167, 85.366...| 40|200|      0|{{46}, 1668708000...|
|{{27.7167, 85.366...| 40|200|      0|{{47}, 1668718800...|
|{{27.7167, 85.366...| 40|200|      0|{{42}, 1668729600...|
+--------------------+---+---+-------+--------------------+
only showing top 5 rows



In [226]:
# LEVEL 1 Unpacking (city and every_3_hr_weather  column)
weather_df=expand_json(weather_df)
weather_df.show(5)

# expand_json function can be replaced by  :  and more ...
# df_weather = df_weather.select("*", F.col("city.*")) \
#     .drop("city")

# df_weather = df_weather.select("*", F.col("coord.*")) \
#     .drop("coord")

city
every_3_hr_weather
+---+---+-------+------------------+-------+-------+--------------+----------+----------+----------+--------+------+----------+-------------------+--------------------+----+------+---+----------+--------------------+-----------------+
|cnt|cod|message|             coord|country|     id|          name|population|   sunrise|    sunset|timezone|clouds|        dt|             dt_txt|                main| pop|  rain|sys|visibility|             weather|             wind|
+---+---+-------+------------------+-------+-------+--------------+----------+----------+----------+--------+------+----------+-------------------+--------------------+----+------+---+----------+--------------------+-----------------+
| 40|200|      0|{27.7167, 85.3667}|     NP|1283647|Baudhatinchule|         0|1668645660|1668684366|   20700|  {42}|1668686400|2022-11-17 12:00:00|{289.69, 868, 62,...|0.32|{0.26}|{n}|     10000|[{light rain, 10n...|{231, 1.19, 1.14}|
| 40|200|      0|{27.7167, 85.3667}|

In [227]:
#explode weather columnn ,it is in list  of dictionary conataing 1 record only
weather_df = weather_df.select ("*", F.explode("weather").alias("weather_dict")).drop("weather")

# LEVEL 2 Unpacking  (coord,clouds,main,rain,sys,wind,weather_dict column)
weather_df=expand_json(weather_df)


# rename column all to coludiness
weather_df = weather_df.withColumnRenamed("all", "cloudiness")
weather_df.show(5)

# weather_df.printSchema()

coord
clouds
main
rain
sys
wind
weather_dict
+---+---+-------+-------+-------+--------------+----------+----------+----------+--------+----------+-------------------+----+----------+-------+-------+----------+----------+----------+--------+--------+---------+------+-------+--------+--------+----+---+---+----+-----+----------------+----+---+------+
|cnt|cod|message|country|     id|          name|population|   sunrise|    sunset|timezone|        dt|             dt_txt| pop|visibility|    lat|    lon|cloudiness|feels_like|grnd_level|humidity|pressure|sea_level|  temp|temp_kf|temp_max|temp_min|  3h|pod|deg|gust|speed|     description|icon| id|  main|
+---+---+-------+-------+-------+--------------+----------+----------+----------+--------+----------+-------------------+----+----------+-------+-------+----------+----------+----------+--------+--------+---------+------+-------+--------+--------+----+---+---+----+-----+----------------+----+---+------+
| 40|200|      0|     NP|1283647|Baudhat

In [228]:
#  delete the column which is not required
weather_df = weather_df.drop("cod","message","cnt","pressure","3h","icon","timezone","population","country","id")

weather_df.printSchema()

root
 |-- name: string (nullable = true)
 |-- sunrise: long (nullable = true)
 |-- sunset: long (nullable = true)
 |-- dt: long (nullable = true)
 |-- dt_txt: string (nullable = true)
 |-- pop: double (nullable = true)
 |-- visibility: long (nullable = true)
 |-- lat: double (nullable = true)
 |-- lon: double (nullable = true)
 |-- cloudiness: long (nullable = true)
 |-- feels_like: double (nullable = true)
 |-- grnd_level: long (nullable = true)
 |-- humidity: long (nullable = true)
 |-- sea_level: long (nullable = true)
 |-- temp: double (nullable = true)
 |-- temp_kf: double (nullable = true)
 |-- temp_max: double (nullable = true)
 |-- temp_min: double (nullable = true)
 |-- pod: string (nullable = true)
 |-- deg: long (nullable = true)
 |-- gust: double (nullable = true)
 |-- speed: double (nullable = true)
 |-- description: string (nullable = true)
 |-- main: string (nullable = true)



## Important functions

In [229]:
# function to convert the unix timestamp to date and time 
def convert_unix_to_date_time(df):
    for c in df.columns:
        if c == "dt":
            df = df.withColumn(c, F.from_unixtime(F.col(c)))
        elif c == "sunrise":
            df = df.withColumn(c, F.from_unixtime(F.col(c)))
        elif c == "sunset":
            df = df.withColumn(c, F.from_unixtime(F.col(c)))
    return df

# convert the unix timestamp to date and time
weather_df = convert_unix_to_date_time(weather_df)

weather_df.show(40)

weather_df.printSchema()

+--------------+-------------------+-------------------+-------------------+-------------------+----+----------+-------+-------+----------+----------+----------+--------+---------+------+-------+--------+--------+---+---+----+-----+----------------+------+
|          name|            sunrise|             sunset|                 dt|             dt_txt| pop|visibility|    lat|    lon|cloudiness|feels_like|grnd_level|humidity|sea_level|  temp|temp_kf|temp_max|temp_min|pod|deg|gust|speed|     description|  main|
+--------------+-------------------+-------------------+-------------------+-------------------+----+----------+-------+-------+----------+----------+----------+--------+---------+------+-------+--------+--------+---+---+----+-----+----------------+------+
|Baudhatinchule|2022-11-17 06:26:00|2022-11-17 17:11:06|2022-11-17 17:45:00|2022-11-17 12:00:00|0.32|     10000|27.7167|85.3667|        42|    289.69|       868|      62|     1014| 290.3|    3.9|   290.3|   286.4|  n|231|1.19| 1.

## Question 1

In [230]:
# save weather_df 'schema to .json file

schema_weather = weather_df.schema # get the schema of the dataframe    
with open('schema_weather.json', 'w') as f:
    json.dump(schema_weather.jsonValue(), f, indent=4)


In [231]:
# load schema_weather from .json file
with open('schema_weather.json', 'r') as f:
    schema_weather_struct = F.StructType.fromJson(json.load(f))

# schema_weather_struct

In [280]:
# save weather_df in schema and payload json format
with open('weather_df.json', 'w') as f:
    for row in weather_df.rdd.collect():
        json.dump({"schema": schema_weather_struct.jsonValue(), "payload": row.asDict()}, f)
        f.write(' ') 
        

### manual schema defination

In [298]:
# define schema manually ## schema extracted from weather_df.json file wont'work
schema_manual= {
    "type": "struct",
    "fields": [
        {
            "field": "name",
            "type": "string",
            "nullable": "true"
        },
        {
            "field": "sunrise",
            "type": "string",
            "nullable": "true"
        },
        {
            "field": "sunset",
            "type": "string",
            "nullable": "true"
        },
        {
            "field": "dt",
            "type": "string",
            "nullable": "true"
        },
        {
            "field": "dt_txt",
            "type": "string",
            "nullable": "true"
        },
        {
            "field": "pop",
            "type": "double",
            "nullable": "true"
        },
        {
            "field": "visibility",
            "type": "int64",
            "nullable": "true"
        },
        {
            "field": "lat",
            "type": "double",
            "nullable": "true"
        },
        {
            "field": "lon",
            "type": "double",
            "nullable": "true"
        },
        {
            "field": "cloudiness",
            "type": "int64",
            "nullable": "true"
        },
        {
            "field": "feels_like",
            "type": "double",
            "nullable": "true"
        },
        {
            "field": "grnd_level",
            "type": "int64",
            "nullable": "true"
        },
        {
            "field": "humidity",
            "type": "int64",
            "nullable": "true"
        },
        {
            "field": "sea_level",
            "type": "int64",
            "nullable": "true"
        },
        {
            "field": "temp",
            "type": "double",
            "nullable": "true"
        },
        {
            "field": "temp_kf",
            "type": "double",
            "nullable": "true"
        },
        {
            "field": "temp_max",
            "type": "double",
            "nullable": "true"
        },
        {
            "field": "temp_min",
            "type": "double",
            "nullable": "true"
        },
        {
            "field": "pod",
            "type": "string",
            "nullable": "true"
        },
        {
            "field": "deg",
            "type": "int64",
            "nullable": "true"
        },
        {
            "field": "gust",
            "type": "double",
            "nullable": "true"
        },
        {
            "field": "speed",
            "type": "double",
            "nullable": "true"
        },
        {
            "field": "description",
            "type": "string",
            "nullable": "true"
        },
        {
            "field": "main",
            "type": "string",
            "nullable": "true"
        }
    ]
}

In [307]:
Producer_save_table = KafkaProducer(bootstrap_servers=[
                         'localhost:9092'], value_serializer=lambda x: dumps(x).encode('utf-8'))
# Producer_save_table = KafkaProducer(bootstrap_servers=['localhost:9092'])

# for row in weather_df.rdd.collect():
#     Producer_save_table.send('cleaned_table', row.asDict())
#     Producer_save_table.flush()  # flush the data to the kafka broker ( topic) and  make sure data  is sent to the kafka broker and  not lost in the buffer



for row in weather_df.rdd.collect():
    # send data to kafka topic in schema and payload json format 
    Producer_save_table.send( 'cleaned_table', {"schema": schema_manual, "payload": row.asDict()})
    Producer_save_table.flush()  # flush the data to the kafka broker ( topic) and  make sure data  is sent to the kafka broker and  not lost in the buffer



In [300]:
# # read and send weather_df.json file to kafka topic
# with open('weather_df.json', 'r') as f:
#     for line in f:
#         Producer_save_table.send('cleaned_table', value=line)
#         Producer_save_table.flush()  # flush the data to the kafka broker ( topic) and  make sure data  is sent to the kafka broker and  not lost in the buffer

In [301]:

# producer_cleaned_table = KafkaProducer(bootstrap_servers=[ 'localhost:9092'], value_serializer=lambda x: dumps(x).encode('utf-8'))

# with open('cleaned_table.json', 'r') as f:
#     for line in f:
#         producer_cleaned_table.send('cleaned_table', line)
#         producer_cleaned_table.flush()  # flush the data to the kafka broker ( topic) and  make sure data  is sent to the kafka broker and  not lost in the buffer

In [308]:

connector_name="cleaned_table-sink-connector"
config={
    "name":connector_name ,
    "config": {
        "connector.class": "io.confluent.connect.jdbc.JdbcSinkConnector",
        "connection.url": "jdbc:postgresql://localhost:5432/sink1",
        "tasks.max": "1",
        "topics": "cleaned_table",
        "insert.mode": "insert",
        "connection.user": "amrit",
        "connection.password": "1234",
        "table.name.format": "cleaned_table",
        "auto.create": "true",
        "auto.evolve": "true",
        "value.converter": "org.apache.kafka.connect.json.JsonConverter",
        "value.converter.schemas.enable": "true",
    }

}


In [309]:
# create sink connector
response = requests.post("http://localhost:8083/connectors", headers={"Content-Type": "application/json"}, data=json.dumps(config))

print(response.json())

{'name': 'cleaned_table-sink-connector', 'config': {'connector.class': 'io.confluent.connect.jdbc.JdbcSinkConnector', 'connection.url': 'jdbc:postgresql://localhost:5432/sink1', 'tasks.max': '1', 'topics': 'cleaned_table', 'insert.mode': 'insert', 'connection.user': 'amrit', 'connection.password': '1234', 'table.name.format': 'cleaned_table', 'auto.create': 'true', 'auto.evolve': 'true', 'value.converter': 'org.apache.kafka.connect.json.JsonConverter', 'value.converter.schemas.enable': 'true', 'name': 'cleaned_table-sink-connector'}, 'tasks': [], 'type': 'sink'}


In [310]:
# connector status=
response = requests.get("http://localhost:8083/connectors/"+connector_name+"/status")
print(response.json())

{'name': 'cleaned_table-sink-connector', 'connector': {'state': 'RUNNING', 'worker_id': '127.0.1.1:8083'}, 'tasks': [{'id': 0, 'state': 'RUNNING', 'worker_id': '127.0.1.1:8083'}], 'type': 'sink'}


In [136]:
#list all connectors
response = requests.get("http://localhost:8083/connectors")
print(response.json())

[]


In [306]:
# delete all the connector
response = requests.get("http://localhost:8083/connectors")
for connector in response.json():
    response = requests.delete("http://localhost:8083/connectors/"+connector)


In [305]:
# delete topic
! kafka-topics --bootstrap-server localhost:9092 --delete --topic cleaned_table